# NYC Apartment Search

_[Project prompt](https://docs.google.com/document/d/1BYVyFBDcTywdUlanH0ysfOrNWPgl7UkqXA7NeewTzxA/edit#heading=h.bpxu7uvknnbk)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add code as you wish._

_**All code below should be consider "pseudo-code" - not functional by itself, and only an idea of a possible approach.**_

## Setup

In [11]:
# All import statements needed for the project, for example:

import json
import pathlib
import urllib.parse

import psycopg2
import pandas as pd
import geoalchemy2 as gdb
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import requests
import shapely
import sqlalchemy as db
from sqlalchemy import create_engine

from sqlalchemy.orm import declarative_base


In [49]:
!pip show sqlalchemy
!pip install --upgrade sqlalchemy

Name: SQLAlchemy
Version: 2.0.21
Summary: Database Abstraction Library
Home-page: https://www.sqlalchemy.org
Author: Mike Bayer
Author-email: mike_mp@zzzcomputing.com
License: MIT
Location: /Users/fate4ever/anaconda3/lib/python3.11/site-packages
Requires: typing-extensions
Required-by: GeoAlchemy2
  Obtaining dependency information for sqlalchemy from https://files.pythonhosted.org/packages/c7/55/d1d2ad054fb7e9188681d56df40ed81c2c198314a805b180b0ec99019da1/SQLAlchemy-2.0.23-cp311-cp311-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 5.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.21
    Uninstalling SQLAlchemy-2.0.21:
      Successfully uninstalled SQLAlchemy-2.0.21


In [16]:
# Any constants you might need; some have been added for you

# Where data files will be read from/written to - this should already exist
DATA_DIR = pathlib.Path("data")
ZIPCODE_DATA_FILE = DATA_DIR / "zipcodes" / "nyc_zipcodes.shp"
ZILLOW_DATA_FILE = DATA_DIR / "zillow_rent_data.csv"

NYC_DATA_APP_TOKEN = "UydgE9GUfZyuG9IpbKml1aKct"
BASE_NYC_DATA_URL = "https://data.cityofnewyork.us/"
NYC_DATA_311 = "erm2-nwe9.geojson"
NYC_DATA_TREES = "5rq2-4hqu.geojson"

DB_NAME = "IE4501_Project_DB"
DB_USER = "postgres"
DB_URL = f"postgresql+psycopg2://{DB_USER}@localhost/{DB_NAME}"
DB_SCHEMA_FILE = "schema.sql"
# directory where DB queries for Part 3 will be saved
QUERY_DIR = pathlib.Path("queries")

In [4]:
# Make sure the QUERY_DIRECTORY exists
if not QUERY_DIR.exists():
    QUERY_DIR.mkdir()

## Part 1: Data Preprocessing

In [4]:
def download_nyc_geojson_data(url, force=False):
    parsed_url = urllib.parse.urlparse(url)
    url_path = parsed_url.path.strip("/")
    
    filename = DATA_DIR / url_path
    
    if force or not filename.exists():
        print(f"Downloading {url} to {filename}...")
        
        ...
        
        with open(filename, "w") as f:
            json.dump(..., f)
        print(f"Done downloading {url}.")

    else:
        print(f"Reading from {filename}...")

    return filename

In [5]:
def load_and_clean_zipcodes(zipcode_datafile):
     # Load the data using geopandas
    gdf = gpd.read_file(ZIPCODE_DATA_FILE)
    
    print(gdf)

    # Drop unnecessary columns. Here, you need to decide which columns are relevant
    # For example, if you only need the zipcode and geometry, you can drop the rest
    columns_to_keep = ['geometry']
    gdf = gdf[columns_to_keep]

    # Remove invalid data points
    # This depends on what you define as invalid. As an example, you might want to remove rows with missing values
    gdf = gdf.dropna()

    # Normalize column names
    # If you want to make column names lowercase for consistency
    gdf.columns = [col.lower() for col in gdf.columns]

    # Normalize data types if needed
    # For example, ensuring ZIPCODE is a string, not a number
    gdf['geometry'] = gdf['geometry'].astype(str)

    # Additional cleaning steps can be added here depending on your specific requirements

    return gdf
    #raise NotImplementedError()

In [6]:
load_and_clean_zipcodes(ZIPCODE_DATA_FILE)

                                              geometry
0    POLYGON ((1038098.252 188138.380, 1038141.936 ...
1    POLYGON ((1001613.713 186926.440, 1002314.243 ...
2    POLYGON ((1011174.276 183696.338, 1011373.584 ...
3    POLYGON ((995908.365 183617.613, 996522.848 18...
4    POLYGON ((991997.113 176307.496, 992042.798 17...
..                                                 ...
258  POLYGON ((950767.507 172848.969, 950787.510 17...
259  POLYGON ((1028453.995 167153.410, 1027813.010 ...
260  POLYGON ((995877.318 203206.075, 995968.511 20...
261  POLYGON ((997731.761 219560.922, 997641.948 21...
262  POLYGON ((986038.661 213051.063, 986135.314 21...

[263 rows x 1 columns]


/var/folders/0h/g9zsw1191fl05wqh_8_l7p_w0000gn/T/ipykernel_30185/339113466.py:22: UserWarning: Geometry column does not contain geometry.
  gdf['geometry'] = gdf['geometry'].astype(str)


,geometry
0,"POLYGON ((1038098.251871 188138.380007, 103814..."
1,"POLYGON ((1001613.712964 186926.439517, 100231..."
2,"POLYGON ((1011174.275536 183696.33771, 1011373..."
3,"POLYGON ((995908.365451 183617.612802, 996522...."
4,"POLYGON ((991997.113431 176307.49586, 992042.7..."
...,...
258,"POLYGON ((950767.506586 172848.968656, 950787...."
259,"POLYGON ((1028453.994913 167153.409838, 102781..."
260,"POLYGON ((995877.318269 203206.074937, 995968...."
261,"POLYGON ((997731.760754 219560.922148, 997641...."


In [1]:
def download_and_clean_311_data():
    import requests

# Your Application Token
    app_token = 'UydgE9GUfZyuG9IpbKml1aKct'

# API endpoints for 311 and tree data
    api_endpoint_311 = 'https://data.cityofnewyork.us/resource/erm2-nwe9.json'


# Headers for authentication
    headers = {
        'X-App-Token': app_token
    }

# Make a GET request to the 311 data endpoint
    response_311 = requests.get(api_endpoint_311, headers=headers)
    data_311 = response_311.json()

# Assuming you have loaded data into DataFrame
    df = pd.DataFrame(data_311)  # Example for 311 data

# Removing unnecessary columns
    columns_to_keep = ['created_date', 'incident_zip', 'latitude','longitude',':@computed_region_efsh_h5xi']  # replace with actual column names
    df = df[columns_to_keep]

# Remove invalid data points
    #df = df[df[':@computed_region_efsh_h5xi'] != null] 
    df = df.dropna()# Replace with your criteria

# Normalize column names
    df.columns = [col.lower().replace(' ', '_') for col in df.columns]

# Convert data types
    #df['date_column'] = pd.to_datetime(df['date_column'])
   # df['numeric_column'] = pd.to_numeric(df['numeric_column'], errors='coerce')

# For geospatial data
   # gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
   # gdf.set_crs("EPSG:4326", inplace=True)  # Replace with current SRID
   # gdf.to_crs("EPSG:NEW_SRID", inplace=True)  # Replace with desired SRID
    
    sampled_df = df.sample(n=10)  # Replace 100 with the number of samples you need
    print(sampled_df)


In [6]:
download_and_clean_311_data()

                created_date incident_zip            latitude  \
231  2023-11-30T23:57:42.000        11226   40.65099388458564   
855  2023-11-30T22:14:35.000        10456   40.82569518319719   
21   2023-12-01T00:54:01.000        10024   40.78270444876873   
932  2023-11-30T22:03:10.000        11211  40.716254989335106   
670  2023-11-30T22:41:32.000        10001   40.75066178373737   
686  2023-11-30T22:38:11.000        11201   40.69340015431026   
208  2023-12-01T00:03:42.000        10456  40.828265237460535   
901  2023-11-30T22:07:02.000        11368  40.744033776595664   
799  2023-11-30T22:22:18.000        11369   40.76415769024426   
779  2023-11-30T22:25:00.000        11205    40.6924668113589   

              longitude :@computed_region_efsh_h5xi  
231  -73.96440523742638                       13510  
855  -73.90661516095976                       10934  
21   -73.97729446204836                       12421  
932  -73.95301450423615                       17613  
670  -74.00343

In [13]:
def download_and_clean_tree_data():
    app_token = 'UydgE9GUfZyuG9IpbKml1aKct'
    api_endpoint_tree = 'https://data.cityofnewyork.us/resource/5rq2-4hqu.geojson'
    headers = {
        'X-App-Token': app_token
    }
    response_tree = requests.get(api_endpoint_tree, headers=headers)
    data_tree = response_tree.json()
    

In [14]:
download_and_clean_tree_data()

In [9]:
def load_and_clean_zillow_data():
   # Load the data using pandas, as we need to handle both non-spatial and spatial data
    df = pd.read_csv(ZILLOW_DATA_FILE)
    print(df)

    # Keep only the necessary columns
    columns_to_keep = ["RegionID", "SizeRank", "RegionName", "StateName", "City", "Metro", "CountyName", "2023-01-31", "2023-08-31"]
    df = df[columns_to_keep]

    # Remove rows with missing values in any of the columns
    df = df.dropna()

    # Normalize column names to lowercase
    df.columns = [col.lower() for col in df.columns]

    # Normalize data types
    # Assuming 'RegionID' and 'SizeRank' should be integers, and dates should be in datetime format
    df['regionid'] = df['regionid'].astype(int)
    df['sizerank'] = df['sizerank'].astype(int)


    # Convert DataFrame to GeoDataFrame
    gdf = gpd.GeoDataFrame(df)

    # Normalize SRID of any geometry if present
    # Example: gdf.set_crs(epsg=YOUR_EPSG_CODE, inplace=True)

    return gdf

In [10]:
load_and_clean_zillow_data()

      RegionID  SizeRank  RegionName RegionType StateName State  \
0        91982         1       77494        zip        TX    TX   
1        91940         3       77449        zip        TX    TX   
2        91733         5       77084        zip        TX    TX   
3        93144         6       79936        zip        TX    TX   
4        62093         7       11385        zip        NY    NY   
...        ...       ...         ...        ...       ...   ...   
6717    418163     30158       89158        zip        NV    NV   
6718     72017     30490       32461        zip        FL    FL   
6719     58956     30490        2876        zip        RI    RI   
6720     91179     30490       76005        zip        TX    TX   
6721     61618     30490       10004        zip        NY    NY   

                   City                                   Metro  \
0                  Katy    Houston-The Woodlands-Sugar Land, TX   
1                  Katy    Houston-The Woodlands-Sugar Land, 

,regionid,sizerank,regionname,statename,city,metro,countyname,2023-01-31,2023-08-31
0,91982,1,77494,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,2027.438438,2053.486247
1,91940,3,77449,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Harris County,1738.217986,1795.384582
2,91733,5,77084,TX,Houston,"Houston-The Woodlands-Sugar Land, TX",Harris County,1706.900064,1757.602011
3,93144,6,79936,TX,El Paso,"El Paso, TX",El Paso County,1458.063897,1488.180414
4,62093,7,11385,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,2895.699421,3064.476503
...,...,...,...,...,...,...,...,...,...
6709,66130,23188,20006,DC,Washington,"Washington-Arlington-Alexandria, DC-VA-MD-WV",District of Columbia,1996.361061,2133.344583
6716,60648,27708,7311,NJ,Jersey City,"New York-Newark-Jersey City, NY-NJ-PA",Hudson County,3762.753868,3950.327047
6717,418163,30158,89158,NV,Las Vegas,"Las Vegas-Henderson-Paradise, NV",Clark County,3509.210744,3310.302151
6720,91179,30490,76005,TX,Arlington,"Dallas-Fort Worth-Arlington, TX",Tarrant County,2169.143026,2383.185013


In [49]:
def load_all_data():
    geodf_zipcode_data = load_and_clean_zipcodes(ZIPCODE_DATA_FILE)
    geodf_311_data = download_and_clean_311_data()
    geodf_tree_data = download_and_clean_tree_data()
    df_zillow_data = load_and_clean_zillow_data()
    return (
        geodf_zipcode_data,
        geodf_311_data,
        geodf_tree_data,
        df_zillow_data
    )

In [51]:
geodf_zipcode_data, geodf_311_data, geodf_tree_data, df_zillow_data = load_all_data()

KeyError: "None of [Index(['Created Date', 'Incident Zip', 'Latitude', 'Longitude', 'Zip Codes'], dtype='object')] are in the [columns]"

In [ ]:
# Show basic info about each dataframe
geodf_zipcode_data.info()

In [ ]:
# Show first 5 entries about each dataframe
geodf_zipcode_data.head()

In [ ]:
geodf_311_data.info()

In [ ]:
geodf_311_data.head()

In [ ]:
geodf_tree_data.info()

In [ ]:
geodf_tree_data.head()

In [ ]:
df_zillow_data.info()

In [ ]:
df_zillow_data.head()

## Part 2: Storing Data

In [11]:
def setup_new_postgis_database(username, db_name):
    

SyntaxError: incomplete input (675149776.py, line 2)

In [5]:
!createdb IE4501_Project_DB

createdb: error: database creation failed: ERROR:  database "IE4501_Project_DB" already exists


In [15]:
!psql -U postgres -d IE4501_Project_DB -f schema.sql

psql: error: schema.sql: No such file or directory


In [6]:
!psql --dbname IE4501_Project_DB -c 'CREATE EXTENSION postgis;'

ERROR:  extension "postgis" already exists


In [17]:
engine = db.create_engine(DB_URL)

NameError: name 'setup_new_postgis_database' is not defined

### Creating Tables


These are just a couple of options to creating your tables; you can use one or the other, a different method, or a combination.

In [ ]:
engine = db.create_engine(DB_URL)

#### Option 1: SQL

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the SQL statements to create your 4 tables
ZIPCODE_SCHEMA = """
TODO
"""

NYC_311_SCHEMA = """
TODO
"""

NYC_TREE_SCHEMA = """
TODO
"""

ZILLOW_SCHEMA = """
TODO
"""

In [ ]:
# create that required schema.sql file
with open(DB_SCHEMA_FILE, "w") as f:
    f.write(ZIPCODE_SCHEMA)
    f.write(NYC_311_SCHEMA)
    f.write(NYC_TREE_SCHEMA)
    f.write(ZILLOW_SCHEMA)

In [ ]:
# If using SQL (as opposed to SQLAlchemy), execute the schema files to create tables
with engine.connect() as connection:
    pass

#### Option 2: SQLAlchemy

In [ ]:
Base = declarative_base()

class Tree(Base):
    __tablename__ = "trees"

    ...


In [ ]:
Base.metadata.create_all(engine)

### Add Data to Database

These are just a couple of options to write data to your tables; you can use one or the other, a different method, or a combination.

#### Option 1: SQL

In [ ]:
def write_dataframes_to_table(tablename_to_dataframe):
    # write INSERT statements or use pandas/geopandas to write SQL
    raise NotImplemented()

In [ ]:
tablename_to_dataframe = {
    "zipcodes": geodf_zipcode_data,
    "complaints": geodf_311_data,
    "trees": geodf_tree_data,
    "rents": df_zillow_data,
}

In [ ]:
write_dataframes_to_table(tablename_to_dataframe)

#### Option 2: SQLAlchemy

In [ ]:
Session = db.orm.sessionmaker(bind=engine)
session = Session()

In [ ]:
for row in geodf_tree_data.iterrows():
    tree = Tree(...)
    session.add(tree)

In [ ]:
session.commit()

## Part 3: Understanding the Data

### Query 1

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    raise NotImplementedError()

In [ ]:
QUERY_1_FILENAME = QUERY_DIR / "FILL_ME_IN"

QUERY_1 = """
FILL_ME_IN
"""

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_1))
    for row in result:
        print(row)

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query your database for the data needed.
    # You can put the data queried into a pandas/geopandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)